In [15]:
import random

# from sentence_transformers import SentenceTransformer
import pandas as pd
# import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import json
import joblib
import requests

In [16]:
state = 'NY'
kmeans = joblib.load(f"{state}_kmeans.pkl")
scaler = joblib.load(f"{state}_scaler.pkl")
weights = [5,5,1,1,1,1,1,1,1]
cols = ['bedrooms', 'bathrooms', 'price', 'livingArea', 'leisure_within_5km', 'shops_within_5km', 'schools_within_5km', 'transit_within_2km', 'homeType']

In [17]:
predict_X=[2,1,1700000,2000,20,400,250,200,2]  # give input here
description = "A large backyard, swimming pool, spacious living room, furniture in bedroom, tiled bathroom"

In [18]:
json_file_path = "./ClusterPoints_Dataset.json"

with open(json_file_path, "r") as file:
    kmeansdata = json.load(file)

json_file_path_1 = "./Labelled_Dataset_with_zpid.json"

with open(json_file_path_1, "r") as file:
    fulldata = json.load(file)

In [19]:
df_predict = pd.DataFrame([predict_X],columns=cols)
scaler_predict=StandardScaler()

df_to_transform_p = df_predict.drop(columns=['homeType'])
df_excluded_p = df_predict[['homeType']]
# Apply the transformation
transformed_array_p = scaler.transform(df_to_transform_p)
print(transformed_array_p)
df_transformed_p = pd.DataFrame(transformed_array_p, columns=df_to_transform_p.columns, index=df_predict.index)
# Recombine the DataFrame
df_result_p = pd.concat([df_transformed_p, df_excluded_p], axis=1)
# print(df_result)
X_p=df_result_p.values
X_p=X_p*weights
X_p

[[ 0.09090596 -0.73108393 -0.20746602  0.8198349  -0.35234394 -0.98294832
   2.22322311 -0.88709456]]


array([[ 0.45452982, -3.65541963, -0.20746602,  0.8198349 , -0.35234394,
        -0.98294832,  2.22322311, -0.88709456,  2.        ]])

In [20]:
predicted_label=kmeans.predict(X_p)
predicted_label

array([2])

In [21]:
state_full_data=fulldata[state]
state_full_data

[{'zpid': 31491229,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 650000.0,
  'homeType': 1,
  'livingArea': 776.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 540,
  'schools_within_5km': 128,
  'transit_within_2km': 121,
  'labels': 3},
 {'zpid': 31492996,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 959000.0,
  'homeType': 1,
  'livingArea': 1000.0,
  'leisure_within_5km': 19,
  'shops_within_5km': 538,
  'schools_within_5km': 127,
  'transit_within_2km': 110,
  'labels': 2},
 {'zpid': 31493494,
  'bedrooms': 1,
  'bathrooms': 1,
  'price': 1100000.0,
  'homeType': 1,
  'livingArea': 815.0,
  'leisure_within_5km': 20,
  'shops_within_5km': 614,
  'schools_within_5km': 141,
  'transit_within_2km': 194,
  'labels': 3},
 {'zpid': 31494186,
  'bedrooms': 0,
  'bathrooms': 1,
  'price': 750000.0,
  'homeType': 1,
  'livingArea': 465.0,
  'leisure_within_5km': 22,
  'shops_within_5km': 717,
  'schools_within_5km': 160,
  'transit_within_2km': 272,
  'labels': 6},
 {'zpid': 31494718

In [22]:
def filter_by_labels(data, labels):
    # Use list comprehension to get dictionaries with matching labels
    filtered_data = [entry for entry in data if entry['labels'] == labels]
    return filtered_data

In [23]:
state_filtered_data = filter_by_labels(state_full_data, predicted_label[0])
len(state_filtered_data)

104

In [24]:
state_filtered_data

[{'zpid': 31492996,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 959000.0,
  'homeType': 1,
  'livingArea': 1000.0,
  'leisure_within_5km': 19,
  'shops_within_5km': 538,
  'schools_within_5km': 127,
  'transit_within_2km': 110,
  'labels': 2},
 {'zpid': 31495264,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 798000.0,
  'homeType': 1,
  'livingArea': 525.0,
  'leisure_within_5km': 28,
  'shops_within_5km': 836,
  'schools_within_5km': 181,
  'transit_within_2km': 268,
  'labels': 2},
 {'zpid': 31499051,
  'bedrooms': 2,
  'bathrooms': 2,
  'price': 3500000.0,
  'homeType': 1,
  'livingArea': 1408.0,
  'leisure_within_5km': 23,
  'shops_within_5km': 632,
  'schools_within_5km': 166,
  'transit_within_2km': 320,
  'labels': 2},
 {'zpid': 31507674,
  'bedrooms': 2,
  'bathrooms': 1,
  'price': 1350000.0,
  'homeType': 1,
  'livingArea': 1053.0,
  'leisure_within_5km': 21,
  'shops_within_5km': 762,
  'schools_within_5km': 182,
  'transit_within_2km': 346,
  'labels': 2},
 {'zpid': 31511

In [25]:
df = pd.DataFrame(state_filtered_data)
df=df.set_index(keys=['zpid'])
df

,bedrooms,bathrooms,price,homeType,livingArea,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,labels
zpid,,,,,,,,,,
31492996,2,2,959000.0,1,1000.000000,19,538,127,110,2
31495264,2,1,798000.0,1,525.000000,28,836,181,268,2
31499051,2,2,3500000.0,1,1408.000000,23,632,166,320,2
31507674,2,1,1350000.0,1,1053.000000,21,762,182,346,2
31511219,2,2,1100000.0,1,963.000000,22,789,197,319,2
...,...,...,...,...,...,...,...,...,...,...
2078380933,2,2,2688000.0,1,1170.000000,29,888,182,244,2
2089719603,2,2,1695000.0,1,1036.000000,21,569,158,242,2
2112868113,2,1,850000.0,1,0.000000,21,744,182,352,2


In [26]:
zpid_list = []
for i in range(len(state_filtered_data)):
    zpid_list.append(state_filtered_data[i]['zpid'])

with open("./data.json", "r") as f:  ## change embeddings file here
    data_embed = json.load(f)
# new_data_point = file_embeddings[str(zpid_list[0])]
print(zpid_list)
dict_embeddings = {}
for i in zpid_list:
    if str(i) in data_embed.keys():
        dict_embeddings[i] = data_embed[str(i)]

[31492996, 31495264, 31499051, 31507674, 31511219, 31520255, 31521020, 31527126, 31528001, 31528555, 31538439, 31545643, 31546758, 31549221, 61968154, 65316519, 69511806, 72527801, 72529433, 72532295, 72534234, 72535178, 72535533, 79498426, 79503309, 79520083, 79984069, 80004512, 80005388, 80007929, 80008463, 80014402, 82784715, 83931345, 83937123, 83940467, 83941177, 89330466, 89331138, 89333534, 89335234, 97496308, 97537243, 97866749, 102960642, 112088524, 112512305, 119911138, 119915660, 119987636, 120226920, 121033987, 121425888, 121999838, 124442587, 125952248, 143092973, 219647993, 219672631, 244701164, 244703189, 244726500, 244746432, 244754141, 244793565, 244801356, 244811132, 244841469, 244850013, 244876319, 244889020, 244889395, 244898624, 244901601, 300195411, 321916980, 342149156, 347350097, 347395167, 347397275, 347397277, 347521090, 347526917, 347682546, 347718880, 347728655, 347753250, 347756370, 347756371, 347762270, 347772687, 2054513617, 2057684716, 2058409968, 206252

In [27]:
payload = {
    'embeddings': dict_embeddings,
    'message': description
}

# Define the URL of the backend server
server_url = "http://127.0.0.1:5000/predict"  # Replace with your server URL

# Send a POST request
response = requests.post(
    server_url,
    json=payload,  # Sending data as JSON
    headers={'Content-Type': 'application/json'}  # Specifying content type
)

# Check if the request was successful
if response.status_code == 200:
    print("Data sent successfully!")
else:
    print("Failed to send data. Status code:", response.status_code)

Data sent successfully!


In [30]:
zpid_sim = response.json()['similarities']
    # print(type(zpid_sim))
json_file_path = "./combined_summary_data.json"

zpid_list = [str(zpid) for zpid, _ in zpid_sim]
with open(json_file_path, "r",encoding="utf-8", errors='ignore') as file:
    data = json.load(file)

# filtered_houses = [house for house in data if str(house["zpid"]) in zpid_list]
filtered_houses = [data[zpid] for zpid in zpid_list if zpid in data]

# Recommendation logic using the selected state
recommended_properties = filtered_houses
recommendations = pd.DataFrame(recommended_properties, index=zpid_list)

In [31]:
recommendations

,bedrooms,bathrooms,price,zipcode,hdpUrl,cityId,homeType,livingArea,livingAreaValue,rentZestimate,...,address.city,address.state,address.zipcode,image_captions,leisure_within_5km,shops_within_5km,schools_within_5km,transit_within_2km,concatenated_desc,summary
112512305,2.0,2.0,1750000,10280,/homedetails/333-Rector-Pl-APT-3E-New-York-NY-...,6181.0,CONDO,1221.0,1221.0,9229.0,...,New York,NY,10280,NaN,20,537,128,117,nan\nOpen city skyline Private lounge area Ren...,Shaker-style wooden cabinetry Abundant natural...
61968154,2.0,1.0,1590000,10075,/homedetails/215-E-80th-St-APT-9A-New-York-NY-...,6181.0,CONDO,884.0,884.0,4500.0,...,New York,NY,10075,a kitchen with a bar and a counter with stools...,24,659,186,300,For the discerning buyer who deserves a brand ...,"A luxury, sophisticated, diamond in the rough...."
2070053469,2.0,2.0,1750000,10036,/homedetails/500-W-45th-St-327-New-York-NY-100...,6181.0,CONDO,1112.0,1112.0,NaN,...,New York,NY,10036,a living room with a glass table and chairs an...,19,585,152,283,Exclusive Offer: Seller covers the first year'...,"2-bedroom, 2-bathroom residence is the first r..."
244703189,2.0,2.0,1600000,10024,/homedetails/250-W-89th-St-APT-2B-New-York-NY-...,6181.0,APARTMENT,0.0,0.0,8269.0,...,New York,NY,10024,a living room with a table and chairs and a co...,20,449,167,204,Award Winning- Notable Designer Home! Rarely a...,"Located in the Penthouse, it offers breathtaki..."
89335234,2.0,2.0,1899000,10005,/homedetails/75-Wall-St-APT-33O-New-York-NY-10...,6181.0,CONDO,1370.0,1370.0,7390.0,...,New York,NY,10005,"a living room with a couch, table, and a kitch...",20,661,155,161,Platinum Properties is proud to present reside...,"This extraordinary 2 bedroom, 2 bathroom is 1,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31546758,2.0,1.0,649000,10029,/homedetails/319-E-105th-St-APT-4F-New-York-NY...,6181.0,CONDO,755.0,755.0,2650.0,...,New York,NY,10029,"a living room with a couch, coffee table, and ...",29,492,212,246,Please call/email/text Sim for an appointment....,"Unit 4-F at The Renaissance East Condominium, ..."
244801356,2.0,2.0,2595000,10028,/homedetails/1025-5th-Ave-10EN-New-York-NY-100...,6181.0,CONDO,0.0,0.0,8295.0,...,New York,NY,10028,a living room with a lot of furniture and a fi...,23,591,187,326,High floor elegant Fifth Avenue home along Mus...,High floor elegant Fifth Avenue home along Mus...
244793565,2.0,1.0,825000,10033,/homedetails/116-Pinehurst-Ave-APT-J52-New-Yor...,6181.0,CONDO,0.0,0.0,2633.0,...,New York,NY,10033,"a living room with a couch, chair, coffee tabl...",41,111,156,99,This Hudson View Gardens crown jewel is unique...,Hudson View Gardens is located on the fifth fl...
347350097,2.0,1.0,520000,10032,/homedetails/544-W-157th-St-1-New-York-NY-1003...,6181.0,CONDO,NaN,NaN,NaN,...,New York,NY,10032,a building with a lot of windows and a car par...,39,227,185,173,Sale may be subject to term & conditions of an...,2 Bedroom co-op on the first floor in Prewar b...
